<a href="https://colab.research.google.com/github/uenoka/testing_for_deep_learnig/blob/Master/fine_tuning/fine_tuning_model_constrlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 共通で使うパスなどの定義
# 共通の変数設定
# 共通フォルダパス
data_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data"
data_imagenet = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data/ImageNet"
data_imagenet_seeds = "/content/gdrive/My Drive/ColabNotebooks/test4ai/data/ImageNet/seeds"
model_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/model"
output_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/output"
tmp_dir = "/content/gdrive/My Drive/ColabNotebooks/test4ai/tmp"


In [2]:
# Goggle Drive つなぐ
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


転移学習のサンプルコードです。もしもKaggleのモデルが利用できない場合、こちらでモデルを作成する必要があるため念のため作成しました。

参考↓

https://www.kaggle.com/rohit1277/cat-dog-classifier-using-vgg16-transfer-learning

In [3]:
# 必要なものインポート
import numpy as np
import pandas as pd
import os
import shutil
from tqdm import tqdm
# モデル構築
import keras
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

Using TensorFlow backend.


In [4]:
# ??
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="/content/gdrive/My Drive/ColabNotebooks/test4ai/data/dog_and_cat/train",target_size=(224,224))


Found 0 images belonging to 2 classes.


In [5]:
# ??
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="/content/gdrive/My Drive/ColabNotebooks/test4ai/data/dog_and_cat/test", target_size=(224,224))


Found 6000 images belonging to 2 classes.


In [0]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [7]:
resnet_model_with_weight_learned = ResNet50(weights='imagenet', include_top=True)
resnet_model_with_weight_no_learned = ResNet50(weights='imagenet', include_top=True)
resnet_model_no_weight_learned = ResNet50(weights=None, include_top=True)

vgg16_model_with_weight_learned = VGG16(weights='imagenet', include_top=True)
vgg16_model_with_weight_no_learned = VGG16(weights='imagenet', include_top=True)
vgg16_model_no_weight_learned = VGG16(weights=None, include_top=True)

vgg19_model_with_weight_learned = VGG19(weights='imagenet', include_top=True)
vgg19_model_with_weight_no_learned = VGG19(weights='imagenet', include_top=True)
vgg19_model_no_weight_learned = VGG19(weights=None, include_top=True)


In [8]:
# 各モデルのレイヤー数を確認（レイヤー構成は同じため代表する一つのみ確認）
resnet_model_with_weight_learned.summary()


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [0]:
vgg16_model_with_weight_learned.summary()


In [0]:
vgg19_model_with_weight_learned.summary()


In [0]:
# 転移学習用関数定義

# 再学習モデルの学習しないレイヤ定義
def inactivate_models(models):
    for model in models:
    # 各モデルの各レイヤーを再学習不可にする
        print(model.name)
        for layers in (model.layers)[:len(model.layers)-4]:
            print(layers)
            layers.trainable = False


def construct_model(models,name):
    for model in models:
    #  for layers in (model.layers)[:19]:
        print(model.name)
        X= model.layers[-2].output
        predictions = Dense(2, activation="softmax")(X)
        final_model = Model(input = model.input, output = predictions)
        final_model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
        train(final_model,name)


In [0]:
def train(model,name):
    model_file_name = model.name + "_" + name + ".h5"
    checkpoint = ModelCheckpoint(model_file_name, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    early = EarlyStopping(monitor='val_acc', min_delta=0, patience=40, verbose=1, mode='auto')
    hist = model.fit_generator(generator= traindata, steps_per_epoch= 2, epochs= 100, validation_data= testdata, validation_steps=1, callbacks=[checkpoint,early])
    model.save_weights(model_dir + model_file_name)


In [0]:

models_with_weight_learned = [
          resnet_model_with_weight_learned, 
          vgg16_model_with_weight_learned, 
          vgg19_model_with_weight_learned, 
          ]
models_with_no_weight_learned = [
          resnet_model_no_weight_learned, 
          vgg16_model_no_weight_learned, 
          vgg19_model_no_weight_learned, 
          ]

models_with_weight_no_learned = [
          resnet_model_with_weight_no_learned, 
          vgg16_model_with_weight_no_learned, 
          vgg19_model_with_weight_no_learned, 
          ]



In [12]:
inactivate_models(models_with_weight_learned)
inactivate_models(models_with_no_weight_learned)

resnet50
vgg16
vgg19
resnet50
vgg16
vgg19


In [0]:
construct_model(models_with_weight_learned, "with_weight")
construct_model(models_with_no_weight_learned, "no_weight")

resnet50


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  app.launch_new_instance()




Epoch 1/100
